In [17]:
import git
from pathlib import Path
import os

ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

DATASET = "coco"
FINAL_DATA_NAME = 'coco-outdoor-wavelet'
CONSTANT_SAMPLE_SIZE = int(1e5)
RAW_DATA_SUFFIX = "coco-outdoor-cropped"
SAVE_DF = False

data_dir = os.path.join(ROOT_DIR, 'raw-data','coco')
file_list = [os.path.join(data_dir, filename) for filename in os.listdir(data_dir)]
file_names = os.listdir(data_dir)
data_dir
BATCH_NUM = None

In [18]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from transform import *
os.chdir(os.path.join(ROOT_DIR, "dataset-preparation"))
freq_df = pd.read_csv(os.path.join(ROOT_DIR, "transformed-data", f"master-frequency-map.csv"), index_col= ["dataset", "transform", "group"])


In [19]:
file_names = [
    filename
    for filename in os.listdir(os.path.join(data_dir, f"{RAW_DATA_SUFFIX}"))
    if not filename.startswith(".")
]

file_list = [
    os.path.join(data_dir, f"{RAW_DATA_SUFFIX}", filename)
    for filename in file_names
]

file_names[:5]


['000000335177.jpg',
 '000000278705.jpg',
 '000000568981.jpg',
 '000000092416.jpg',
 '000000173830.jpg']

In [20]:
'''Assuming No batching is required. Not applicable for agriVision'''

# data_dir = os.path.join(ROOT_DIR, "raw-data", "agriVision", "full-agriVision-RGB-cleaned")

# for channel in ['red', 'blue', 'green', 'gray', 'infrared']:

#     channel_fr = convert_to_fourier_basis(data_dir, channel, debug = True)
#     pd.to_pickle(channel_fr, os.path.join(ROOT_DIR, "transformed-data", f"full-agriVision-fourier-{channel}-df.pickle"))

#     min_group, max_group = 2, max(channel_fr['band'])
#     group_data_map = dict()
#     group_data_map_size = dict()
#     for group in np.arange(min_group, max_group + 1):
#         data = channel_fr[(channel_fr['band'] == group)]['data'].iloc[0]
#         group_data_map[group] = np.sort(data)[np.round(np.linspace(0, data.size - 1, min(data.size, CONSTANT_SAMPLE_SIZE))).astype(int)] 
#         group_data_map_size[group] = data.size
    
#     pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME}-{channel}.pickle"))
#     pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME}-{channel}-size.pickle"))
    

'Assuming No batching is required. Not applicable for agriVision'

In [21]:
'''To split large dataset into many batches, only needs to be run once'''
# k = 10000
# target_dir = os.path.join(ROOT_DIR, 'raw-data', 'agriVision') # Where the batch{i} folders will be created
# directorySplit(folder_dir = data_dir, target_dir = target_dir, name = RAW_DATA_SUFFIX, k = k)
# print(f"{len(file_names)//k} batches created" )

'To split large dataset into many batches, only needs to be run once'

In [22]:
'''Show all subsets of data in raw data folder that have already been created'''
print(''.join([x+"\n" for x in os.listdir(data_dir) if x.__contains__(RAW_DATA_SUFFIX)]))


coco-outdoor-cropped



In [23]:
import numpy as np
np.random.seed(42)  
def jpg_opener(path):
    
    # Apply jitter
    image = np.array(Image.open(path).convert('RGB'))
    arr = image.astype(np.float64)
    jitter = np.random.uniform(-0.5, 0.5, arr.shape)
    arr += jitter
    arr = arr - np.mean(arr)
    arr = arr / np.std(arr)

    
    return arr


# Wavelet

In [24]:
FINAL_DATA_NAME = 'coco-outdoor-wavelet'
if BATCH_NUM is None:
    batch_dir = os.path.join(ROOT_DIR, "raw-data", "coco", f"{RAW_DATA_SUFFIX}")
else:
    batch_dir = os.path.join(ROOT_DIR, "raw-data", "coco", f"batch{BATCH_NUM}-{RAW_DATA_SUFFIX}")

In [25]:
import gc

TRANSFORM = "wavelet"
channel = "red"

channel_wv_full = convert_to_wavelet_basis(batch_dir, channel, debug=True, image_opener=jpg_opener)
channel_wv_full['data'] = channel_wv_full['data'].apply(lambda x: x.astype(np.float32))  # if needed, or skip if you jittered already

for orientation_label in ['horizontal', 'vertical', 'diagonal']:
    FINAL_DATA_NAME_ORIENTED = f"{FINAL_DATA_NAME}-{orientation_label}"
    orientation_code = 'H' if orientation_label == 'horizontal' else ('V' if orientation_label == 'vertical' else 'D')

    channel_wv = channel_wv_full[channel_wv_full['orientation'] == orientation_code].copy()

    if SAVE_DF:
        df_save_path = os.path.join(
            ROOT_DIR,
            "transformed-data",
            f"dataframes/{'' if BATCH_NUM is None else f'batch{BATCH_NUM}'}{FINAL_DATA_NAME_ORIENTED}-{channel}-df.pickle"
        )
        pd.to_pickle(channel_wv, df_save_path)

    min_group, max_group = 2, max(channel_wv['layer'])
    group_data_map = {}
    group_data_map_size = {}

    for group in np.arange(min_group, max_group + 1):
        filtered = channel_wv[channel_wv['layer'] == group]
        if filtered.empty:
            continue

        data = filtered['data'].iloc[0]
        sampled = np.sort(data)[np.round(np.linspace(0, data.size - 1, min(data.size, CONSTANT_SAMPLE_SIZE))).astype(int)]

        group_data_map[group] = sampled
        group_data_map_size[group] = data.size

        freq_df.loc[f"{DATASET}-outdoor", TRANSFORM, group] = channel_wv[(channel_wv['layer'] == group) & (channel_wv['orientation'] == orientation_code)]['frequency'].iloc[0]

    freq_df_save_path = os.path.join(ROOT_DIR, "transformed-data", "master-frequency-map.csv")
    freq_df.to_csv(freq_df_save_path)

    group_save_base = os.path.join(
        ROOT_DIR,
        "transformed-data",
        f"{'' if BATCH_NUM is None else f'batch{BATCH_NUM}'}{FINAL_DATA_NAME_ORIENTED}-{channel}"
    )
    pd.to_pickle(group_data_map, f"{group_save_base}.pickle")
    pd.to_pickle(group_data_map_size, f"{group_save_base}-size.pickle")

    del channel_wv
    del group_data_map
    del group_data_map_size
    gc.collect()

del channel_wv_full
gc.collect()

9 layers being used


  0%|          | 0/2446 [00:00<?, ?it/s]

0

In [26]:
import gc

TRANSFORM = "wavelet"
channel = "green"

channel_wv_full = convert_to_wavelet_basis(batch_dir, channel, debug=True, image_opener=jpg_opener)
channel_wv_full['data'] = channel_wv_full['data'].apply(lambda x: x.astype(np.float32))  # if needed, or skip if you jittered already

for orientation_label in ['horizontal', 'vertical', 'diagonal']:
    FINAL_DATA_NAME_ORIENTED = f"{FINAL_DATA_NAME}-{orientation_label}"
    orientation_code = 'H' if orientation_label == 'horizontal' else ('V' if orientation_label == 'vertical' else 'D')

    channel_wv = channel_wv_full[channel_wv_full['orientation'] == orientation_code].copy()

    if SAVE_DF:
        df_save_path = os.path.join(
            ROOT_DIR,
            "transformed-data",
            f"dataframes/{'' if BATCH_NUM is None else f'batch{BATCH_NUM}'}{FINAL_DATA_NAME_ORIENTED}-{channel}-df.pickle"
        )
        pd.to_pickle(channel_wv, df_save_path)

    min_group, max_group = 2, max(channel_wv['layer'])
    group_data_map = {}
    group_data_map_size = {}

    for group in np.arange(min_group, max_group + 1):
        filtered = channel_wv[channel_wv['layer'] == group]
        if filtered.empty:
            continue

        data = filtered['data'].iloc[0]
        sampled = np.sort(data)[np.round(np.linspace(0, data.size - 1, min(data.size, CONSTANT_SAMPLE_SIZE))).astype(int)]

        group_data_map[group] = sampled
        group_data_map_size[group] = data.size

        freq_df.loc[f"{DATASET}-outdoor", TRANSFORM, group] = channel_wv[(channel_wv['layer'] == group) & (channel_wv['orientation'] == orientation_code)]['frequency'].iloc[0]

    freq_df_save_path = os.path.join(ROOT_DIR, "transformed-data", "master-frequency-map.csv")
    freq_df.to_csv(freq_df_save_path)

    group_save_base = os.path.join(
        ROOT_DIR,
        "transformed-data",
        f"{'' if BATCH_NUM is None else f'batch{BATCH_NUM}'}{FINAL_DATA_NAME_ORIENTED}-{channel}"
    )
    pd.to_pickle(group_data_map, f"{group_save_base}.pickle")
    pd.to_pickle(group_data_map_size, f"{group_save_base}-size.pickle")

    del channel_wv
    del group_data_map
    del group_data_map_size
    gc.collect()

del channel_wv_full
gc.collect()

9 layers being used


  0%|          | 0/2446 [00:00<?, ?it/s]

0

In [27]:
TRANSFORM = "wavelet"
channel = "blue"

for orientation_label in ['horizontal', 'vertical', 'diagonal']:
    FINAL_DATA_NAME_ORIENTED = f"{FINAL_DATA_NAME}-{orientation_label}"
    orientation_code = 'H' if orientation_label == 'horizontal' else ('V' if orientation_label == 'vertical' else 'D')

    channel_wv = convert_to_wavelet_basis(batch_dir, channel, debug=True, image_opener=jpg_opener)
    channel_wv['data'] = channel_wv['data'].apply(lambda x: x.astype(np.float32))

    if SAVE_DF:
        if BATCH_NUM is None:
            pd.to_pickle(channel_wv, os.path.join(ROOT_DIR, "transformed-data", f"dataframes/{FINAL_DATA_NAME_ORIENTED}-{channel}-df.pickle"))
        else:
            pd.to_pickle(channel_wv, os.path.join(ROOT_DIR, "transformed-data", f"dataframes/batch{BATCH_NUM}{FINAL_DATA_NAME_ORIENTED}-{channel}-df.pickle"))

    min_group, max_group = 2, max(channel_wv['layer'])
    group_data_map = dict()
    group_data_map_size = dict()

    for group in np.arange(min_group, max_group + 1):
        data = channel_wv[(channel_wv['orientation'] == orientation_code) & (channel_wv['layer'] == group)]['data'].iloc[0]
        group_data_map[group] = np.sort(data)[np.round(np.linspace(0, data.size - 1, min(data.size, CONSTANT_SAMPLE_SIZE))).astype(int)]
        group_data_map_size[group] = data.size

        freq_df.loc[f"{DATASET}-outdoor", TRANSFORM, group] = channel_wv[(channel_wv['layer'] == group) & (channel_wv['orientation'] == orientation_code)]['frequency'].iloc[0]

    print(f"printing {orientation_code}")
    freq_df.to_csv(os.path.join(ROOT_DIR, "transformed-data", "master-frequency-map.csv"))

    if BATCH_NUM is None:
        pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME_ORIENTED}-{channel}.pickle"))
        pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME_ORIENTED}-{channel}-size.pickle"))
    else:
        pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME_ORIENTED}-{channel}.pickle"))
        pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME_ORIENTED}-{channel}-size.pickle"))

    print(group_data_map)

    del channel_wv
    del group_data_map
    del group_data_map_size


9 layers being used


  0%|          | 0/2446 [00:00<?, ?it/s]

printing H
{np.int64(2): array([-248.90155, -245.36464, -240.90932, ...,  293.33658,  298.61066,
        299.80692], dtype=float32), np.int64(3): array([-167.18323, -152.58598, -150.61557, ...,  165.4808 ,  168.26833,
        182.62132], dtype=float32), np.int64(4): array([-109.05787, -103.1326 ,  -95.81834, ...,   93.97632,   95.14471,
         95.29361], dtype=float32), np.int64(5): array([-63.653046, -59.018433, -56.84846 , ...,  62.445503,  62.448654,
        68.18389 ], dtype=float32), np.int64(6): array([-42.406116, -34.80475 , -32.94273 , ...,  31.922071,  36.267544,
        51.16247 ], dtype=float32), np.int64(7): array([-30.279846, -17.994648, -15.848033, ...,  16.025124,  18.073732,
        30.375885], dtype=float32), np.int64(8): array([-19.936314 ,  -9.324538 ,  -8.121117 , ...,   7.954714 ,
         8.9942045,  18.58975  ], dtype=float32), np.int64(9): array([-10.291904 ,  -4.7899375,  -4.332287 , ...,   4.436077 ,
         4.809806 ,  10.307971 ], dtype=float32)}
9 layers

  0%|          | 0/2446 [00:00<?, ?it/s]

printing V
{np.int64(2): array([-253.36737, -227.91344, -206.6585 , ...,  199.48708,  210.34825,
        246.52722], dtype=float32), np.int64(3): array([-158.11023, -148.67575, -147.95576, ...,  143.52937,  156.29349,
        161.45753], dtype=float32), np.int64(4): array([-107.16327,  -92.89132,  -92.40085, ...,   89.03467,   93.17929,
         96.12743], dtype=float32), np.int64(5): array([-94.84072 , -87.82645 , -72.58377 , ...,  58.032738,  80.52294 ,
        88.55304 ], dtype=float32), np.int64(6): array([-59.684437, -33.984097, -29.280413, ...,  31.943926,  34.86555 ,
        54.70654 ], dtype=float32), np.int64(7): array([-39.286835 , -18.862453 , -15.704175 , ...,  14.8732195,
        16.488783 ,  34.083508 ], dtype=float32), np.int64(8): array([-24.133783 ,  -8.9532995,  -7.874131 , ...,   7.801162 ,
         8.658029 ,  22.261492 ], dtype=float32), np.int64(9): array([-15.566425 ,  -4.6508923,  -4.212825 , ...,   4.228297 ,
         4.747013 ,  11.425196 ], dtype=float32)}
9 

  0%|          | 0/2446 [00:00<?, ?it/s]

printing D
{np.int64(2): array([-143.45427 , -119.454346, -106.550514, ...,  140.64656 ,
        142.94087 ,  143.51741 ], dtype=float32), np.int64(3): array([-103.41609 ,  -96.28206 ,  -82.891335, ...,   74.88598 ,
         75.224945,   75.32754 ], dtype=float32), np.int64(4): array([-92.85886 , -84.8201  , -78.82225 , ...,  52.922165,  68.119835,
        69.65954 ], dtype=float32), np.int64(5): array([-36.15082 , -33.806767, -33.43674 , ...,  36.846966,  42.016113,
        64.110146], dtype=float32), np.int64(6): array([-24.23961  , -18.931164 , -16.820726 , ...,  15.9465475,
        17.288595 ,  39.431095 ], dtype=float32), np.int64(7): array([-14.110894 ,  -8.95307  ,  -8.191431 , ...,   8.0649185,
         9.101522 ,  26.043062 ], dtype=float32), np.int64(8): array([-11.341499 ,  -5.12177  ,  -4.5671954, ...,   4.7249055,
         5.227389 ,  16.235075 ], dtype=float32), np.int64(9): array([-7.2737083, -3.0344403, -2.7567074, ...,  2.826076 ,  3.1373591,
        8.5096   ], dtype=

In [28]:
TRANSFORM = "wavelet"
channel = "gray"

for orientation_label in ['horizontal', 'vertical', 'diagonal']:
    FINAL_DATA_NAME_ORIENTED = f"{FINAL_DATA_NAME}-{orientation_label}"
    orientation_code = 'H' if orientation_label == 'horizontal' else ('V' if orientation_label == 'vertical' else 'D')

    channel_wv = convert_to_wavelet_basis(batch_dir, channel, debug=True, image_opener=jpg_opener)
    channel_wv['data'] = channel_wv['data'].apply(lambda x: x.astype(np.float32))

    if SAVE_DF:
        if BATCH_NUM is None:
            pd.to_pickle(channel_wv, os.path.join(ROOT_DIR, "transformed-data", f"dataframes/{FINAL_DATA_NAME_ORIENTED}-{channel}-df.pickle"))
        else:
            pd.to_pickle(channel_wv, os.path.join(ROOT_DIR, "transformed-data", f"dataframes/batch{BATCH_NUM}{FINAL_DATA_NAME_ORIENTED}-{channel}-df.pickle"))

    min_group, max_group = 2, max(channel_wv['layer'])
    group_data_map = dict()
    group_data_map_size = dict()

    for group in np.arange(min_group, max_group + 1):
        data = channel_wv[(channel_wv['orientation'] == orientation_code) & (channel_wv['layer'] == group)]['data'].iloc[0]
        group_data_map[group] = np.sort(data)[np.round(np.linspace(0, data.size - 1, min(data.size, CONSTANT_SAMPLE_SIZE))).astype(int)]
        group_data_map_size[group] = data.size

        freq_df.loc[f"{DATASET}-outdoor", TRANSFORM, group] = channel_wv[(channel_wv['layer'] == group) & (channel_wv['orientation'] == orientation_code)]['frequency'].iloc[0]

    print(f"printing {orientation_code}")
    freq_df.to_csv(os.path.join(ROOT_DIR, "transformed-data", "master-frequency-map.csv"))

    if BATCH_NUM is None:
        pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME_ORIENTED}-{channel}.pickle"))
        pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"{FINAL_DATA_NAME_ORIENTED}-{channel}-size.pickle"))
    else:
        pd.to_pickle(group_data_map, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME_ORIENTED}-{channel}.pickle"))
        pd.to_pickle(group_data_map_size, os.path.join(ROOT_DIR, "transformed-data", f"batch{BATCH_NUM}{FINAL_DATA_NAME_ORIENTED}-{channel}-size.pickle"))

    print(group_data_map)

    del channel_wv
    del group_data_map
    del group_data_map_size


9 layers being used


  0%|          | 0/2446 [00:00<?, ?it/s]

printing H
{np.int64(2): array([-227.59982, -211.6978 , -210.49812, ...,  229.1469 ,  234.18005,
        238.40411], dtype=float32), np.int64(3): array([-142.19849, -137.54417, -136.22075, ...,  137.66188,  139.92137,
        141.57138], dtype=float32), np.int64(4): array([-95.80921 , -94.41179 , -88.96685 , ...,  88.473885,  90.45101 ,
        93.88421 ], dtype=float32), np.int64(5): array([-63.615456, -56.390774, -50.290874, ...,  54.9296  ,  55.25739 ,
        61.6705  ], dtype=float32), np.int64(6): array([-43.240005, -35.257442, -31.822956, ...,  31.03447 ,  33.0974  ,
        49.784573], dtype=float32), np.int64(7): array([-30.36581 , -17.515657, -15.701508, ...,  15.706691,  17.286062,
        30.372635], dtype=float32), np.int64(8): array([-19.952301,  -9.414831,  -8.218139, ...,   8.009918,   9.059657,
        18.63883 ], dtype=float32), np.int64(9): array([-10.270793 ,  -4.8279467,  -4.365114 , ...,   4.41231  ,
         4.8522735,  10.274315 ], dtype=float32)}
9 layers being

  0%|          | 0/2446 [00:00<?, ?it/s]

printing V
{np.int64(2): array([-193.79915, -184.71513, -176.9553 , ...,  177.59071,  197.27577,
        220.88876], dtype=float32), np.int64(3): array([-142.54572, -135.99301, -135.63663, ...,  128.74712,  131.31953,
        139.80336], dtype=float32), np.int64(4): array([-106.64207 , -102.447365,  -91.85571 , ...,   81.47305 ,
         83.80761 ,   90.04442 ], dtype=float32), np.int64(5): array([-92.154335, -65.598816, -58.26406 , ...,  55.05353 ,  69.57895 ,
        93.31011 ], dtype=float32), np.int64(6): array([-41.82805 , -29.757128, -28.484272, ...,  29.394611,  33.60498 ,
        54.712566], dtype=float32), np.int64(7): array([-40.265476, -17.244188, -15.240449, ...,  14.597781,  16.641548,
        35.759506], dtype=float32), np.int64(8): array([-24.200768 ,  -8.858701 ,  -7.703463 , ...,   7.8136024,
         8.682833 ,  22.19258  ], dtype=float32), np.int64(9): array([-15.67546  ,  -4.6746054,  -4.2294154, ...,   4.248662 ,
         4.7725115,  11.443029 ], dtype=float32)}
9 

  0%|          | 0/2446 [00:00<?, ?it/s]

printing D
{np.int64(2): array([-129.89288 , -118.982994,  -98.228096, ...,  119.52885 ,
        134.98183 ,  135.53291 ], dtype=float32), np.int64(3): array([-103.20113 ,  -81.690254,  -74.07149 , ...,   73.30292 ,
         76.473404,   77.14178 ], dtype=float32), np.int64(4): array([-93.29486 , -89.575455, -61.62243 , ...,  48.849945,  51.99479 ,
        54.1991  ], dtype=float32), np.int64(5): array([-34.516552, -32.202614, -29.490135, ...,  33.822174,  34.23632 ,
        66.57307 ], dtype=float32), np.int64(6): array([-23.221298, -17.56351 , -15.928752, ...,  15.538143,  16.341173,
        39.381996], dtype=float32), np.int64(7): array([-14.61431  ,  -9.1739435,  -8.103491 , ...,   8.029601 ,
         9.002452 ,  26.069544 ], dtype=float32), np.int64(8): array([-11.375231 ,  -5.1655784,  -4.6163864, ...,   4.7349577,
         5.2524242,  16.050257 ], dtype=float32), np.int64(9): array([-7.4578094, -3.0576026, -2.7782264, ...,  2.83364  ,  3.1448357,
        8.486573 ], dtype=float3